# Feature angineering

**Purpose of script:**

Creating new features 

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
# paths
df_path = r"../Data/combined/"

out_path = r"../Data/combined/"

In [3]:
# data load 
#df = pd.read_csv(df_path + 'melt_2019-07-01.csv')
df = pd.read_parquet(df_path + 'melt_2019-07-01.parquet.gzip')

In [4]:
df

,x,y,mw_value,opt_value,col,row,v1,v2,v3,v4,v5,v6,v7,v8,v9
0,186500.0,-678500.0,0.0,0.016349,823,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,187500.0,-678500.0,0.0,0.000484,824,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,185500.0,-679500.0,0.0,0.000385,822,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,186500.0,-679500.0,0.0,0.188975,823,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,187500.0,-679500.0,0.0,0.236826,824,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752685,71500.0,-3268500.0,0.0,0.017829,708,2606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
752686,66500.0,-3269500.0,0.0,0.015932,703,2607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
752687,64500.0,-3270500.0,0.0,0.014779,701,2608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
752688,65500.0,-3270500.0,0.0,0.014123,702,2608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Features:

In script 05:
- Column and row numbers
- Neighboring pixels
- Water pixels from mw removed 
- Masked pixels from opt removed


Date

In [5]:
df['date'] = '2019-07-01'
df["date"]= pd.to_datetime(df["date"])
# df['month'] = df["date"].dt.month

Aggregated/pooled values

In [6]:
# delet v5? same as mw value

In [7]:
cols = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9']

In [8]:
df[cols] = df[cols].replace(-1, np.NaN) # to skip -1 when calculating mean
df['mean'] = df[cols].mean(axis = 1) # mean value of 9 pixels around

Distance from margin/shore

In [ ]:
# add if coast column - if at least one na but not all 

Geohash

In [ ]:
# convert x and y to useful sections of geodata

Write

In [9]:
df.to_parquet(out_path + 'melt_2019-07-01_extended.parquet.gzip', index= False)